In [1]:
import json
import os
import logging
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [64]:
CUSTOMER_ID = os.environ.get("CUSTOMER_ID")
CORPUS_ID = os.environ.get("CORPUS_ID")
API_KEY = os.environ.get("API_KEY")
AUTH_URL = os.environ.get("AUTH_URL")
APP_CLIENT_ID = os.environ.get("APP_CLIENT_ID")
APP_CLIENT_SECRET = os.environ.get("APP_CLIENT_SECRET")
IDX_ADDRESS = os.environ.get("IDX_ADDRESS")
TONIC_VALIDATE_API_KEY = os.environ.get("TONIC_VALIDATE_API_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

## Create Corpus

In [ ]:
def _get_create_corpus_json():
    """Returns a create corpus json."""
    corpus = {
        "name": "Vectara Test Corpus(Python)",
        "description": "An example corpus generated via REST API from Python code.",
    }
    return json.dumps({"corpus": corpus})


def create_corpus(customer_id: int, admin_address: str, jwt_token: str):
    """Create a corpus.
    Args:
        customer_id: Unique customer ID in vectara platform.
        admin_address: Address of the admin server. e.g., api.vectara.io
        jwt_token: A valid Auth token.

    Returns:
        (response, True) in case of success and returns (error, False) in case of failure.
    """

    post_headers = {
        "customer-id": f"{customer_id}",
        "Authorization": f"Bearer {jwt_token}",
    }
    response = requests.post(
        f"https://{admin_address}/v1/create-corpus",
        data=_get_create_corpus_json(),
        verify=True,
        headers=post_headers,
    )

    if response.status_code != 200:
        logging.error(
            "Create Corpus failed with code %d, reason %s, text %s",
            response.status_code,
            response.reason,
            response.text,
        )
        return response, False

    message = response.json()
    if message["status"] and message["status"]["code"] != "OK":
        logging.error("Create Corpus failed with status: %s", message["status"])
        return message["status"], False

    return message, True

## Delete Corpus

In [ ]:
def _get_delete_corpus_json(customer_id: int, corpus_id: int):
    """Returns a delete corpus JSON."""
    corpus = {
        "customer_id": customer_id,
        "corpus_id": corpus_id,
    }

    return json.dumps(corpus)


def delete_corpus(customer_id: int, corpus_id: int, admin_address: str, jwt_token: str):
    """Deletes a corpus.

    Args:
        customer_id: Unique customer ID in vectara platform.
        corpus_id: Corpus ID in vectara platform.
        admin_address: Address of the admin server. e.g., api.vectara.io
        jwt_token: A valid Auth token.

    Returns:
        (response, True) in case of success and returns (error, False) in case of failure.
    """
    post_headers = {
        "customer-id": f"{customer_id}",
        "Authorization": f"Bearer {jwt_token}",
    }
    response = requests.post(
        f"https://{admin_address}/v1/delete-corpus",
        data=_get_delete_corpus_json(customer_id, corpus_id),
        verify=True,
        headers=post_headers,
    )

    if response.status_code != 200:
        logging.error(
            "Delete Corpus failed with code %d, reason %s, text %s",
            response.status_code,
            response.reason,
            response.text,
        )
        return response, False

    message = response.json()
    if message["status"] and message["status"]["code"] != "OK":
        logging.error("Delete Corpus failed with status: %s", message.status)
        return message.status, False

    return message, True

## Reset Corpus

In [ ]:
def _get_reset_corpus_json(customer_id: int, corpus_id: int):
    """Returns a reset corpus json."""
    corpus = {
        "customer_id": customer_id,
        "corpus_id": corpus_id,
    }

    return json.dumps(corpus)


def reset_corpus(customer_id: int, corpus_id: int, admin_address: str, jwt_token: str):
    """Reset a corpus.
    Args:
        customer_id: Unique customer ID in vectara platform.
        corpus_id: Corpus ID in vectara platform.
        admin_address: Address of the admin server. e.g., api.vectara.io
        jwt_token: A valid Auth token.

    Returns:
        (response, True) in case of success and returns (error, False) in case of failure.
    """

    post_headers = {
        "customer-id": f"{customer_id}",
        "Authorization": f"Bearer {jwt_token}",
    }
    response = requests.post(
        f"https://{admin_address}/v1/reset-corpus",
        data=_get_reset_corpus_json(customer_id, corpus_id),
        verify=True,
        headers=post_headers,
    )

    if response.status_code != 200:
        logging.error(
            "Reset Corpus failed with code %d, reason %s, text %s",
            response.status_code,
            response.reason,
            response.text,
        )
        return response, False

    message = response.json()
    if message["status"] and message["status"]["code"] != "OK":
        logging.error("Delete Corpus failed with status: %s", message.status)
        return message.status, False

    return message, True

## File Upload

In [3]:
def get_jwt_token():
    """Get JWT token from authentication service."""
    auth_url = AUTH_URL
    client_id = APP_CLIENT_ID
    client_secret = APP_CLIENT_SECRET

    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
    }

    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    response = requests.post(auth_url, headers=headers, data=data)

    if response.status_code == 200:
        response_data = response.json()
        return response_data.get("access_token")
    else:
        print("Error:", response.text)
        return None

In [102]:
def upload_file(
    customer_id: int, corpus_id: int, idx_address: str, jwt_token: str, file_path: str
):
    """Uploads a file to the corpus.

    Args:
        customer_id: Unique customer ID in vectara platform.
        corpus_id: ID of the corpus to which data needs to be indexed.
        idx_address: Address of the indexing server. e.g., api.vectara.io
        jwt_token: A valid Auth token.
        file_path: Path to the file to be uploaded.

    Returns:
        (response, True) in case of success and returns (error, False) in case of failure.
    """

    # Extract filename from the file path
    filename = os.path.basename(file_path)

    post_headers = {"Authorization": f"Bearer {jwt_token}"}
    with open(file_path, "rb") as file:
        response = requests.post(
            f"https://{idx_address}/v1/upload?c={customer_id}&o={corpus_id}",
            files={"file": (file.name, file, "application/octet-stream")},
            data={"doc_metadata": f'{{"filename": "{filename}"}}'},
            verify=True,
            headers=post_headers,
        )

    if response.status_code != 200:
        logging.error(
            "REST upload failed with code %d, reason %s, text %s",
            response.status_code,
            response.reason,
            response.text,
        )
        return response, False

    message = response.json()["response"]
    # An empty status indicates success.
    if message["status"] and message["status"]["code"] not in ("OK", "ALREADY_EXISTS"):
        logging.error("REST upload failed with status: %s", message["status"])
        return message["status"], False

    return message, True


def upload_files_in_directory(
    customer_id: int, corpus_id: int, idx_address: str, directory_path: str
):
    """Uploads all files in a directory to the corpus.

    Args:
        customer_id: Unique customer ID in Vectara platform.
        corpus_id: ID of the corpus to which data needs to be indexed.
        idx_address: Address of the indexing server. e.g., api.vectara.io
        directory_path: Path to the directory containing files to be uploaded.

    Returns:
        A list of tuples containing (response, success) for each file upload.
    """
    jwt_token = get_jwt_token()
    if not jwt_token:
        return []

    file_uploads = []
    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)
        if os.path.isfile(file_path):
            response, success = upload_file(
                customer_id, corpus_id, idx_address, jwt_token, file_path
            )
            file_uploads.append((response, success))
    return file_uploads

In [108]:
upload_files_in_directory(
    customer_id=CUSTOMER_ID,
    corpus_id=CORPUS_ID,
    idx_address="api.vectara.io",
    directory_path="corpus",
)

[({'status': {},
   'quotaConsumed': {'numChars': '2126', 'numMetadataChars': '887'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '2271', 'numMetadataChars': '840'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '2262', 'numMetadataChars': '845'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '2321', 'numMetadataChars': '948'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '2049', 'numMetadataChars': '737'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '2093', 'numMetadataChars': '894'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '3344', 'numMetadataChars': '1024'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '2278', 'numMetadataChars': '840'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '2161', 'numMetadataChars': '835'}},
  True),
 ({'status': {},
   'quotaConsumed': {'numChars': '2121', 'numMetadataChars': '842'}},
  True),
 ({'status': {},
   'quotaConsumed': {'

In [106]:
abs_temp_dir = os.path.abspath("temp")
upload_files_in_directory(
    customer_id=CUSTOMER_ID,
    corpus_id=CORPUS_ID,
    idx_address="api.vectara.io",
    directory_path=abs_temp_dir,
)

[({'status': {'code': 'ALREADY_EXISTS'},
   'quotaConsumed': {'numChars': '1996', 'numMetadataChars': '896'}},
  True)]

## Query Corpus

In [38]:
# Dictionary mapping language names to their lowercase initials
language_initials = {
    "English": "eng",
    "German": "deu",
    "French": "fra",
    "Chinese": "zho",
    "Korean": "kor",
    "Arabic": "ara",
    "Russian": "rus",
    "Thai": "tha",
    "Dutch": "nld",
    "Italian": "ita",
    "Portuguese": "por",
    "Spanish": "spa",
    "Japanese": "jpn",
    "Polish": "pol",
    "Turkish": "tur",
    "Vietnamese": "vie",
    "Indonesian": "ind",
    "Czech": "ces",
    "Ukrainian": "ukr",
    "Greek": "ell",
    "Hebrew": "heb",
    "Farsi/Persian": "fas",
    "Hindi": "hin",
    "Urdu": "urd",
    "Swedish": "swe",
    "Bengali": "ben",
    "Malay": "msa",
    "Romanian": "ron",
}

models = {
    "GPT-3.5-Turbo": "vectara-summary-ext-v1.2.0",
    "GPT-4-Turbo": "vectara-summary-ext-v1.3.0",
}

In [55]:
def _get_query_json(
    customer_id: int,
    corpus_id: int,
    query_value: str,
    summarizer_prompt_name,
    response_lang,
    top_k=5,
    max_summarized_results=10,
    lambda_val=0.025,
):
    """Returns a query JSON."""
    query = {
        "query": [
            {
                "query": query_value,
                "num_results": top_k,
                "corpus_key": [
                    {
                        "customer_id": customer_id,
                        "corpus_id": corpus_id,
                        "lexicalInterpolationConfig": {"lambda": lambda_val},
                    }
                ],
                "summary": [
                    {
                        "summarizerPromptName": summarizer_prompt_name,  # vectara-summary-ext-v1.2.0 (gpt-3.5-turbo) vectara-summary-ext-v1.3.0 (gpt-4.0)
                        "responseLang": response_lang,  # auto to auto-detect
                        "maxSummarizedResults": max_summarized_results,
                        "factual_consistency_score": True,
                    }
                ],
            },
        ],
    }
    return json.dumps(query)


def query_corpus(
    customer_id: int, corpus_id: int, query_address: str, jwt_token: str, query: str
):
    """Queries the data.

    Args:
        customer_id: Unique customer ID in vectara platform.
        corpus_id: ID of the corpus to which data needs to be indexed.
        query_address: Address of the querying server. e.g., api.vectara.io
        jwt_token: A valid Auth token.

    Returns:
        (response, True) in case of success and returns (error, False) in case of failure.

    """
    post_headers = {
        "customer-id": f"{customer_id}",
        "Authorization": f"Bearer {jwt_token}",
    }

    response = requests.post(
        f"https://{query_address}/v1/query",
        data=_get_query_json(
            customer_id,
            corpus_id,
            query,
            models["GPT-4-Turbo"],
            language_initials["English"],
        ),
        verify=True,
        headers=post_headers,
    )

    if response.status_code != 200:
        logging.error(
            "Query failed with code %d, reason %s, text %s",
            response.status_code,
            response.reason,
            response.text,
        )
        return response, False

    message = response.json()
    if message["status"] and any(
        status["code"] != "OK" for status in message["status"]
    ):
        logging.error("Query failed with status: %s", message["status"])
        return message["status"], False

    responses = message["responseSet"][0]["response"]
    documents = message["responseSet"][0]["document"]
    summary = message["responseSet"][0]["summary"][0]["text"]
    factual_consistency_score = message["responseSet"][0]["summary"][0][
        "factualConsistency"
    ]["score"]

    res = [[r["text"], r["score"]] for r in responses]
    return res, summary, factual_consistency_score, documents

In [68]:
query_txt = "which was John Doe's condition?"
results, summary, score, documents = query_corpus(
    CUSTOMER_ID, CORPUS_ID, IDX_ADDRESS, get_jwt_token(), query_txt
)
summary

'John Doe, born on May 17, 1985, visited Sunrise Health Medical Center due to severe, intermittent abdominal pain and persistent nausea he had been experiencing over the last 48 hours [2]. His primary diagnosis was chronic kidney disease, stage 3 [3]. His physical examination results indicated a stable chronic condition [4]. His blood pressure was slightly high at 140/90 mmHg [4], which was being managed with ACE inhibitors [3]. He was also referred to a nephrologist for ongoing management of his condition and a dietitian for consultation on kidney health [3]. His condition was expected to be managed with potential stabilization [3]. His known allergies include penicillin, and he has a past medical history of asthma and high blood pressure [2].'

In [57]:
score

0.7441291

In [73]:
len(documents)

3

## Validate With Tonic Validate

In [61]:
from tonic_validate import ValidateScorer, Benchmark, LLMResponse, ValidateApi
from tonic_validate.metrics import (
    AnswerSimilarityMetric,
    RetrievalPrecisionMetric,
    AugmentationPrecisionMetric,
    AugmentationAccuracyMetric,
    AnswerConsistencyMetric,
)
import time
import datetime
from tqdm import tqdm

In [65]:
from tonic_validate import ValidateApi

validate_api = ValidateApi(TONIC_VALIDATE_API_KEY)
benchmark = validate_api.get_benchmark("fff9af14-643e-4bfd-b565-e8761c1dca88")

In [67]:
benchmark.items

[BenchmarkItem(question='What was the followup plan for Charlotte Gonzalez', answer='Neurology follow-up after MRI results', benchmark_id=None),
 BenchmarkItem(question='Why did Amelia Sanchez visit the hospital?', answer='Routine check-up for celiac disease management.', benchmark_id=None),
 BenchmarkItem(question='Does Amelia Sanchez have any known allergies?', answer='Gluten', benchmark_id=None),
 BenchmarkItem(question='Which doctor took care of Benjamin Lee', answer='Dr. Richard Kim', benchmark_id=None),
 BenchmarkItem(question="What is Amelia Sanchez's date of birth?", answer='October 23, 1989', benchmark_id=None),
 BenchmarkItem(question='Which doctor took care of Amelia Sanchez?', answer='Dr. Anita Gupta with qualifications in MD, Gastroenterology ', benchmark_id=None),
 BenchmarkItem(question='What is the medical history of Benjamin Lee', answer='Chronic kidney disease ', benchmark_id=None),
 BenchmarkItem(question='What was the treatment plan for Amelia Sanchez?', answer='Med

In [74]:
def get_rag_response(query_txt):
    results, summary, score, documents = (
        query_corpus(CUSTOMER_ID, CORPUS_ID, IDX_ADDRESS, get_jwt_token(), query_txt),
    )
    context_list = [doc["metadata"][6]["value"] for doc in documents]
    return {"llm_answer": summary, "llm_context_list": context_list}


# Score the responses
print("Scoring responses...")
scorer = ValidateScorer(
    [
        AnswerSimilarityMetric(),
        RetrievalPrecisionMetric(),
        AugmentationPrecisionMetric(),
        AugmentationAccuracyMetric(),
        AnswerConsistencyMetric(),
    ],
    model_evaluator="gpt-3.5-turbo",
    fail_on_error=True,
)
run = scorer.score(benchmark, get_rag_response)
validate_api = ValidateApi(TONIC_VALIDATE_API_KEY)
validate_api.upload_run(
    "145ee5b0-4e7b-4abd-aeac-243b59274926",
    run,
    run_metadata={
        "run_name": "Init Run",
        "top_k": 5,
        "summarized_top_k": 10,
    },
)
print("Done!")

Scoring responses...


Retrieving responses:   0%|          | 0/21 [00:15<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 1)

## Query With LlamaIndex

In [117]:
load_dotenv()
os.environ["VECTARA_CUSTOMER_ID"] = os.environ.get("CUSTOMER_ID")
os.environ["VECTARA_CORPUS_ID"] = "6"
os.environ["VECTARA_API_KEY"] = os.environ.get("CORPUS_6_API_KEY")

In [119]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.indices.managed.vectara import VectaraIndex

index = VectaraIndex()
query_engine = index.as_query_engine()
response = query_engine.query("What was John Doe's condition?")
response.response

LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.


"John Doe's condition was diagnosed as chronic kidney disease, stage 3. He presented complaints of severe abdominal pain, nausea, and occasional dizziness, with slightly elevated creatinine levels. The treatment plan included ACE inhibitors for blood pressure management and a referral to a nephrologist for ongoing care. The prognosis involved potential stabilization of the condition with follow-up scheduled in three months. The specialist recommended continued nephrology follow-up and diet consultation for kidney health [2]."

In [ ]:
os.environ["VECTARA_CORPUS_ID"] = "7"
os.environ["VECTARA_API_KEY"] = os.environ.get("CORPUS_7_API_KEY")

In [101]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.indices.managed.vectara import VectaraIndex

documents = SimpleDirectoryReader("corpus").load_data()
index = VectaraIndex().from_documents(documents)

LLM is explicitly disabled. Using MockLLM.
Embeddings have been explicitly disabled. Using MockEmbedding.


In [97]:
# Get an answer to the query based on the content of the essay
response = index.as_query_engine(
    similarity_top_k=5,
)
answer = response.query("What was John Doe's condition?")
answer

Response(response='', source_nodes=[], metadata=None)

In [88]:
response = index.as_query_engine().query("What was John Doe's condition?")
print(response)

None


In [1]:
import os
from together import Together

TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")
client = Together(api_key=TOGETHER_API_KEY)

In [2]:
response = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    messages=[
        {
            "role": "user", 
            "content": "hello"
         }
    ],
)
response.choices[0].message.content

APIError: Error code: 524 - Error code: 524 -<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>


<title>api.together.xyz | 524: A timeout occurred</title>
<meta charset="UTF-8" />
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
<meta http-equiv="X-UA-Compatible" content="IE=Edge" />
<meta name="robots" content="noindex, nofollow" />
<meta name="viewport" content="width=device-width,initial-scale=1" />
<link rel="stylesheet" id="cf_styles-css" href="/cdn-cgi/styles/main.css" />


</head>
<body>
<div id="cf-wrapper">
    <div id="cf-error-details" class="p-0">
        <header class="mx-auto pt-10 lg:pt-6 lg:px-8 w-240 lg:w-full mb-8">
            <h1 class="inline-block sm:block sm:mb-2 font-light t

"The city that never sleeps! New York City is a vibrant and exciting destination that offers something for everyone. Here are some fun things to do in New York:\n\n**Iconic Landmarks and Attractions:**\n\n1. Visit the Statue of Liberty and Ellis Island: Take a ferry to Liberty Island to see up close the iconic statue and learn about its history at the nearby Ellis Island Immigration Museum.\n2. Explore the Empire State Building: Enjoy panoramic views of the city from the observation deck of this iconic skyscraper.\n3. Walk across the Brooklyn Bridge: Take in the stunning views of the Manhattan skyline and the East River while walking across this historic bridge.\n4. Visit the 9/11 Memorial & Museum: A poignant tribute to the victims of the 9/11 attacks, featuring two massive reflecting pools surrounded by the names of those who were killed.\n\n**Museums and Galleries:**\n\n1. The Metropolitan Museum of Art (The Met): One of the world's largest and most famous museums, with a collection

In [2]:
from openai import OpenAI
api_key = os.environ.get("OPENAI_API_KEY") 
client = OpenAI(api_key=api_key,)
chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "hello"}],
    model="gpt-3.5-turbo",
)
chat_completion.choices[0].message.content

'Hello! How are you today?'